In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import time
from tqdm import tqdm

from google.oauth2.service_account import Credentials
import pygsheets
import googleapiclient
import logging
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
import gslides
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

import firebase_admin
from firebase_admin import credentials, firestore
#pip install google-api-core==1.32.0
#pip install protobuf==3.15.5
#pip install google-cloud-firestore==2.3.4


###>>>>>>>>>>>>>>>>>>>>>>>>>> Firestore credentials
try:
    app = firebase_admin.get_app()
except ValueError as e:
    cred = credentials.Certificate("C:/Users/leemn/Documents/Loop Service Account/accountkey_firebase.json")
    firebase_admin.initialize_app(cred)
db = firestore.client() 


# Nandos SA - zVznneXPn3Zaj75n5Q3N
categoryDocRef = db.collection('clients').document('zVznneXPn3Zaj75n5Q3N')
file = db.collection('drivers').where('client', '==', categoryDocRef)
drivers_docs = [doc.to_dict() for doc in file.stream()]

drivers_df = pd.DataFrame.from_dict(drivers_docs)

drivers_df.delivery_permissions = drivers_df.delivery_permissions.str[0].str.get('hub_id')
drivers_df['hub_id'] = drivers_df['delivery_permissions']

drivers_df.active_trip = drivers_df.delivery_permissions.str[0].str.get('trip_id')
drivers_df['active_trip'] = drivers_df['active_trip']
                                 
final_df_drivers = drivers_df[['created_at','name','employee_code','vehicle_type',
                                'mobile_no','email','blocked','lunch','hub_id',
                                'idle_since','on_active_trip','active_trip','last_trip_rejection',
                                'primary_employer','scheduled_lunch','available','sos']]





###>>>>>>>>>>>>>>> GBQ credentials
credentials_gbq = Credentials.from_service_account_file('C:/Users/leemn/Documents/Loop Service Account/cb-prod-297913-623cea238b9a.json')

hub_id_list = list(final_df_drivers.hub_id)
hub_id_list = [x for x in hub_id_list if pd.notnull(x)]
p = tuple(hub_id_list)
query_drivers = '''
                SELECT DISTINCT hub_id, hub_name, branch.name AS branch_name
                FROM `cb-prod-297913.cb_prod_analytics.cb_receive_trip_orders` 
                WHERE client_id IN ('zVznneXPn3Zaj75n5Q3N')
                AND hub_id IN {}
                GROUP BY 1,2,3      
                '''.format(p)
branch_drivers = pd.read_gbq(query_drivers, project_id='cb-prod-297913',
                                 dialect='standard',
                                 credentials=credentials_gbq)


####>>>>>>>Final DF
final_df = pd.merge(final_df_drivers, branch_drivers, how='left', on=['hub_id'])

final_df['blocked'] = np.where(final_df['blocked']==False,'No',
                              (np.where(final_df['blocked']==True,'Yes',final_df['blocked'])))

final_df['lunch'] = np.where(final_df['lunch']==False,'No',
                               (np.where(final_df['lunch']==True,'Yes',final_df['lunch'])))

final_df['on_active_trip'] = np.where(final_df['on_active_trip']==False,'No',
                               (np.where(final_df['on_active_trip']==True,'Yes',final_df['on_active_trip'])))

final_df['scheduled_lunch'] = np.where(final_df['scheduled_lunch']==False,'No',
                               (np.where(final_df['scheduled_lunch']==True,'Yes',final_df['scheduled_lunch'])))

final_df['sos'] = np.where(final_df['sos']==False,'No',
                               (np.where(final_df['sos']==True,'Yes',final_df['sos'])))

final_df['available'] = np.where(final_df['available']==False,'No',
                               (np.where(final_df['available']==True,'Yes',final_df['available'])))

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Push final table to BigQuery >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
final_df.to_gbq(destination_table = 'cb-prod-297913.cb_prod_analytics.cb_fleet_management',
              project_id = 'cb-prod-297913',
              credentials = credentials_gbq,
              chunksize = 50000,
              progress_bar = True,
              if_exists = 'replace')

100%|██████████| 1/1 [00:00<?, ?it/s]
